<a href="https://colab.research.google.com/github/emmafengxi/Detect-Medicare-Fraud/blob/master/Extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# authenticate access to google cloud bigquery
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
# access to google drive
from google.colab import drive

drive.mount('/content/drive')
print('Authenticated')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Authenticated


In [0]:
import pandas as pd
import numpy as np

In [0]:
# define project
project_id = 'medicare-springboard-capstone'
proj = bigquery.Client(project=project_id)

# Explore LEIE: the list of excluded providers

---


*   Can be fraud or violations of Medicare/Medicaid
*   Can be individuals or entities




In [0]:
df_LEIE = pd.read_csv('/content/drive/My Drive/Data Science/Springboard/Data/LEIE.csv')
df_LEIE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72476 entries, 0 to 72475
Data columns (total 18 columns):
LASTNAME      69365 non-null object
FIRSTNAME     69366 non-null object
MIDNAME       72476 non-null object
BUSNAME       3108 non-null object
GENERAL       72476 non-null object
SPECIALTY     68276 non-null object
UPIN          6333 non-null object
NPI           72476 non-null int64
DOB           68499 non-null float64
ADDRESS       72467 non-null object
CITY          72475 non-null object
STATE         72471 non-null object
ZIP           72476 non-null int64
EXCLTYPE      72476 non-null object
EXCLDATE      72476 non-null int64
REINDATE      72476 non-null int64
WAIVERDATE    72476 non-null int64
WVRSTATE      13 non-null object
dtypes: float64(1), int64(5), object(12)
memory usage: 10.0+ MB


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df_LEIE.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN


In [0]:
# from the provider dataset below, the following are the top 4 states in terms of the number of providers: 'CA', 'FL', 'TX', 'NY'
# extract data from these 4 states and 'TN': 
df_LEIE_5s = df_LEIE[df_LEIE.STATE.isin(['CA', 'FL', 'TX', 'NY', 'TN'])]
df_LEIE_5s.shape

# save data to google drive
df_LEIE_5s.to_csv('LEIE_5S.csv')
!cp LEIE_5S.csv /content/drive/My\ Drive/Data\ Science/Springboard/Data

In [0]:
# extract TN data
df_LEIE_TN = df_LEIE_5s[df_LEIE_5s.STATE == 'TN']
print(df_LEIE_TN.shape)

# save data to google drive
df_LEIE_TN.to_csv('LEIE_TN.csv')
!cp LEIE_TN.csv /content/drive/My\ Drive/Data\ Science/Springboard/Data

(1691, 18)


# Explore the provider data

---

In [0]:
%%bigquery --project medicare-springboard-capstone
SELECT *
  FROM `bigquery-public-data.medicare.physicians_and_other_supplier_2014`
 LIMIT 5


,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
0,1780752352,PREMIER SURGERY CENTER OF SANTA MARIA,,,,,O,816 EAST ENOS DRIVE,B,SANTA MARIA,93454,CA,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,66.0,42,66,1914.900000,8901.515151,1500.324242,1314.514242
1,1578567954,"E. RONALD SALVITTI, M.D., INC.",,,,,O,750 E BEAU ST,,WASHINGTON,153016661,PA,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,25.0,14,25,1566.240000,2000.000000,1227.930000,1308.466800
2,1821074485,"SURGERY CENTER OF FARMINGTON, L.L.C.",,,,,O,400 PARKLAND DR,,FARMINGTON,636402906,MO,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,51.0,32,51,1491.230000,7456.150000,1169.120000,1315.470000
3,1770581829,"PHYSICIANS' SURGERY CENTER, LLC",,,,,O,3840 W 75TH ST,,PRAIRIE VILLAGE,662084126,KS,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,14.0,11,14,1563.564286,11471.250000,1225.840714,1315.470714
4,1043216617,TRIAD EYE INSTITUTE PLLC,,,,,O,3131 MILITARY BLVD,,MUSKOGEE,744012290,OK,US,Ambulatory Surgical Center,Y,F,0191T,Internal insertion of eye fluid drainage device,N,34.0,23,34,1485.280000,2275.000000,1100.073235,1334.536471


In [0]:
# found a country variable, check any providers outsider of US?
%%bigquery --project medicare-springboard-capstone
SELECT nppes_provider_country AS providers_country,
       COUNT(*) AS total_providers_per_country
  FROM `bigquery-public-data.medicare.physicians_and_other_supplier_2014`
 GROUP BY nppes_provider_country
 # I will omit providers outsider of US

,providers_country,total_providers_per_country
0,VE,13
1,GR,8
2,KR,35
3,CA,29
4,AE,9
5,IT,6
6,KW,6
7,IL,4
8,AU,8
9,JP,56


In [0]:
# check counts of providers for each state
%%bigquery --project medicare-springboard-capstone
SELECT nppes_provider_state AS providers_state,
       COUNT(*) AS total_providers_per_state
  FROM `bigquery-public-data.medicare.physicians_and_other_supplier_2014`
 WHERE nppes_provider_country = 'US'
 GROUP BY nppes_provider_state
 ORDER BY 2 DESC
 LIMIT 10
# I will limit my analysis to the following states: CA, FL, TX, NY, TN

,providers_state,total_providers_per_state
0,CA,722404
1,FL,686363
2,TX,634518
3,NY,596114
4,PA,412966
5,IL,383620
6,OH,342838
7,NC,337521
8,MI,333817
9,NJ,317227


In [0]:
df_p2014 = proj.query('''
  SELECT *
    FROM `bigquery-public-data.medicare.physicians_and_other_supplier_2014`
   WHERE nppes_provider_state IN ('TN')
  ''').to_dataframe()

In [0]:
df_p2014.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255311 entries, 0 to 255310
Data columns (total 26 columns):
npi                                 255311 non-null int64
nppes_provider_last_org_name        255311 non-null object
nppes_provider_first_name           255311 non-null object
nppes_provider_mi                   255311 non-null object
nppes_credentials                   255311 non-null object
nppes_provider_gender               255311 non-null object
nppes_entity_code                   255311 non-null object
nppes_provider_street1              255311 non-null object
nppes_provider_street2              255311 non-null object
nppes_provider_city                 255311 non-null object
nppes_provider_zip                  255311 non-null object
nppes_provider_state                255311 non-null object
nppes_provider_country              255311 non-null object
provider_type                       255311 non-null object
medicare_participation_indicator    255311 non-null object
place

In [0]:
df_p2014.to_csv('Provider2014_TN.csv', index=False)
!cp Provider2014_TN.csv /content/drive/My\ Drive/Data\ Science/Springboard/Data

In [0]:
df_p2014 = proj.query('''
  SELECT *
    FROM `bigquery-public-data.medicare.physicians_and_other_supplier_2014`
   WHERE nppes_provider_state IN ('CA', 'FL', 'TX', 'NY', 'TN')
  ''').to_dataframe()

df_p2014.to_csv('Provider2014_5S.csv', index=False)
!cp Provider2014_5S.csv /content/drive/My\ Drive/Data\ Science/Springboard/Data